In [1]:
import numpy
import bokeh.plotting as bplot
bplot.output_notebook()
from matplotlib import pyplot
%matplotlib notebook

BokehJS successfully loaded.

In [2]:
from soapy import WFS, confParse
from aotools import circle

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/qt.py:13: ShimWarning: The `IPython.qt` package has been deprecated. You should import from qtconsole instead.
  "You should import from qtconsole instead.", ShimWarning)


In [3]:
SOAPY_CONF = "conf/test_conf.py"

In [4]:
# load Soapy config
config = confParse.Configurator(SOAPY_CONF)
config.loadSimParams()
config.wfss[0].pxlsPerSubap = 30
mask = circle.circle(config.sim.pupilSize/2., config.sim.simSize)

In [5]:
# One arcec in tilt Amp across scope
tilt_rad = 1./3600 * numpy.pi/180
tilt_amp = tilt_rad*config.tel.telDiam/2.
tilt_amp/(550e-9)

35.25917680796625

In [10]:
PXL_SCALE = 0.2 #asec/pxl

In [20]:
# Measure current pixel scale
maxTilt = 10*PXL_SCALE # arcsecs
tilts_asec = numpy.linspace(-0.5*maxTilt, 0.5*maxTilt, 10)

tilts_rad = (tilts_asec/3600.) * (numpy.pi/180)
# Must convert arcsecs of tilt to amp in nm
tiltAmps = 1e9 * (tilts_rad * config.tel.telDiam)# in nm amp

# Make a unit tilt to multiply
coords = numpy.linspace(-0.5, 0.5, config.sim.pupilSize)
coords = numpy.pad(coords, pad_width=[config.sim.simPad,]*2, mode="constant")
tilt = numpy.meshgrid(coords, coords)[0]

measuredTilts = numpy.zeros((tiltAmps.shape[0]))
for i, A in enumerate(tiltAmps):
    config.wfss[0].subapFOV = config.wfss[0].pxlsPerSubap*PXL_SCALE
    wfs = WFS.ShackHartmann(config.sim, config.wfss[0], config.atmos, config.lgss[0], mask)
    
    aTilt = tilt*A
    slopes = wfs.frame(aTilt)
    measuredTilts[i] = -1*slopes[wfs.activeSubaps:].mean()

In [21]:
A = numpy.vstack([tilts_asec, numpy.ones(len(tilts_asec))]).T
pxlScale, bias = numpy.linalg.lstsq(A, measuredTilts)[0]
print("Measured Pixel Scale: {} pxls/arcsec".format(1./pxlScale))

Measured Pixel Scale: 0.199434900846 pxls/arcsec


In [22]:
plt = bplot.figure()
plt.line(tilts_asec, measuredTilts)
plt.line(tilts_asec, pxlScale*tilts_asec + bias)
bplot.show(plt)